## Rainfall & Temp Preprocessing

In [3]:
!pip install pandas 

  Using cached pandas-1.4.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.7 MB)
     |████████████████████████████████| 17.1 MB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 500 kB 85.8 MB/s eta 0:00:01


In [4]:
import pandas as pd
import glob
import os

In [40]:
root_wd = '/home/studio-lab-user/sagemaker-studiolab-notebooks/'

### Step 1: Glob files together

In [46]:
rainfall_files = glob.glob(os.path.join(root_wd,"data/weather/raw/worldbank_precipitation_1901_2021_africa/pr*.csv"))
rainfall_file_list = []
for f in rainfall_files: 
    df_ = pd.read_csv(f, skiprows=2)
    df_['country_code'] = os.path.basename(f)[36:-4]
    rainfall_file_list.append(df_)
rainfall_df = pd.concat(rainfall_file_list, ignore_index=True)

In [52]:
rainfall_df.head(5)

,Unnamed: 0,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,country_code
0,1901,141.31,138.39,148.28,183.71,81.15,11.44,4.30,15.11,48.08,83.07,145.15,131.09,BDI
1,1902,116.93,114.32,131.94,158.42,83.25,35.45,3.84,17.26,63.80,86.54,145.57,123.85,BDI
2,1903,140.76,122.02,167.95,244.21,95.04,21.04,4.28,11.13,59.32,107.98,167.14,138.98,BDI
3,1904,134.92,155.71,167.79,144.12,84.78,8.69,3.44,20.28,43.79,83.77,145.21,163.91,BDI
4,1905,148.16,110.46,194.70,182.10,62.55,8.83,3.19,12.20,60.97,83.45,172.30,180.59,BDI


In [66]:
temp_files = glob.glob(os.path.join(root_wd,"data/weather/raw/worldbank_temp_1901_2021_africa/tas*.csv"))
temp_file_list = []
for f in temp_files: 
    df_ = pd.read_csv(f, skiprows=2)
    df_['country_code'] = os.path.basename(f)[37:-4]
    temp_file_list.append(df_)
temp_df = pd.concat(temp_file_list, ignore_index=True)

In [53]:
temp_df.head(5)

,Unnamed: 0,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,country_code
0,1901,19.97,20.08,20.14,20.20,19.84,18.52,19.09,20.09,20.92,21.31,20.66,20.77,BDI
1,1902,19.97,20.09,20.14,20.12,21.33,20.80,19.11,21.28,20.80,21.03,19.76,19.07,BDI
2,1903,19.97,20.15,19.74,20.10,19.85,19.98,19.11,21.28,20.79,20.72,19.97,19.87,BDI
3,1904,19.47,19.51,19.73,19.80,19.97,19.80,19.30,21.27,21.06,19.97,19.51,19.39,BDI
4,1905,19.97,19.91,19.57,19.08,19.61,19.51,18.84,19.94,20.83,21.18,19.85,19.16,BDI


### Step 2: Clean column names

In [54]:
## Map Alpha-3 country code to country name and Alpha-2 country code
africa_country_mapping = pd.read_csv(os.path.join(root_wd,"data/africa_countries.csv"))
mapping_dict = dict(africa_country_mapping[['Alpha-3 code', 'Country']].values)
mapping_dict2 = dict(africa_country_mapping[['Alpha-3 code', 'Alpha-2 code']].values)

In [55]:
mapping_dict

{'DZA': 'Algeria',
 'AGO': 'Angola',
 'BEN': 'Benin',
 'BWA': 'Botswana',
 'BFA': 'Burkina Faso',
 'BDI': 'Burundi',
 'CPV': 'Cabo Verde',
 'CMR': 'Cameroon',
 'CAF': 'Central African Republic (the)',
 'TCD': 'Chad',
 'COM': 'Comoros (the)',
 'COG': 'Congo (Republic of)',
 'CIV': "Côte d'Ivoire",
 'DJI': 'Djibouti',
 'COD': 'Congo (Democratic Republic of)',
 'EGY': 'Egypt',
 'GNQ': 'Equatorial Guinea',
 'ERI': 'Eritrea',
 'SWZ': 'Eswatini',
 'ETH': 'Ethiopia',
 'GAB': 'Gabon',
 'GMB': 'Gambia (the)',
 'GHA': 'Ghana',
 'GIN': 'Guinea',
 'GNB': 'Guinea-Bissau',
 'KEN': 'Kenya',
 'LSO': 'Lesotho',
 'LBR': 'Liberia',
 'LBY': 'Libya',
 'MDG': 'Madagascar',
 'MWI': 'Malawi',
 'MLI': 'Mali',
 'MRT': 'Mauritania',
 'MUS': 'Mauritius',
 'MAR': 'Morocco',
 'MOZ': 'Mozambique',
 'NAM': 'Namibia',
 'NER': 'Niger (the)',
 'NGA': 'Nigeria',
 'RWA': 'Rwanda',
 'STP': 'Sao Tome and Principe',
 'SEN': 'Senegal',
 'SYC': 'Seychelles',
 'SLE': 'Sierra Leone',
 'SOM': 'Somalia',
 'ZAF': 'South Africa',
 '

In [72]:
rainfall_df = rainfall_df.rename(columns={'Unnamed: 0':'Year', 'country_code':'country_cd_alpha3'})
temp_df = temp_df.rename(columns={'Unnamed: 0':'Year','country_code':'country_cd_alpha3'})

rainfall_df['country_name'] = rainfall_df['country_cd_alpha3'].map(mapping_dict)
temp_df['country_name'] = temp_df['country_cd_alpha3'].map(mapping_dict)

rainfall_df['country_cd_alpha2'] = rainfall_df['country_cd_alpha3'].map(mapping_dict2)
temp_df['country_cd_alpha2'] = temp_df['country_cd_alpha3'].map(mapping_dict2)

rainfall_df = rainfall_df.reindex(columns=['country_name','country_cd_alpha3','country_cd_alpha2','Year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec'])
temp_df = temp_df.reindex(columns=['country_name','country_cd_alpha3','country_cd_alpha2','Year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec'])

rainfall_df = rainfall_df.sort_values(['country_name','Year']).reset_index(drop=True)
temp_df = temp_df.sort_values(['country_name','Year']).reset_index(drop=True)

In [73]:
rainfall_df.head(5)

,country_name,country_cd_alpha3,country_cd_alpha2,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,Algeria,DZA,DZ,1901,10.58,10.65,8.67,5.65,10.10,2.65,3.35,6.14,9.38,19.25,10.96,14.36
1,Algeria,DZA,DZ,1902,5.46,10.30,9.18,6.82,7.17,3.51,3.81,4.52,7.76,15.36,11.64,12.66
2,Algeria,DZA,DZ,1903,8.28,3.42,7.66,5.60,4.90,4.48,2.25,2.98,6.55,7.13,8.74,16.66
3,Algeria,DZA,DZ,1904,23.82,12.87,15.78,12.49,2.62,4.61,2.25,3.82,5.70,8.52,7.05,11.53
4,Algeria,DZA,DZ,1905,8.67,6.00,5.48,6.84,12.88,4.31,2.90,3.51,3.98,16.45,8.96,13.39


In [74]:
temp_df.head(5)

,country_name,country_cd_alpha3,country_cd_alpha2,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,Algeria,DZA,DZ,1901,12.22,14.36,18.22,23.06,26.27,31.83,32.80,32.34,29.10,23.10,17.58,12.77
1,Algeria,DZA,DZ,1902,11.97,15.45,18.50,22.89,25.92,30.46,32.88,32.40,29.02,23.19,17.82,13.18
2,Algeria,DZA,DZ,1903,12.60,14.93,18.41,22.08,26.33,30.39,32.21,31.95,28.90,24.00,17.64,13.03
3,Algeria,DZA,DZ,1904,12.08,15.26,18.16,22.38,26.77,31.10,32.86,32.38,28.83,23.72,17.53,13.43
4,Algeria,DZA,DZ,1905,11.71,13.93,18.56,22.83,26.24,30.90,32.57,32.13,29.03,23.28,17.75,12.97


In [75]:
print(rainfall_df['country_name'].nunique())
print(temp_df['country_name'].nunique())

54
54


In [ ]:
rainfall_df.to_csv(os.path.join(root_wd,"data/africa_countries.csv")), index=False)
temp_df.to_csv('/content/drive/MyDrive/ASDI/rainfall_temp/africa_temp_cleaned.csv', index=False)